In [ ]:
# import os
# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"

In [ ]:
# stdlib
import os

environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
environment

In [ ]:
# syft absolute
import syft as sy

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    reset=True,
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
high_client = sy.login(
    url="http://localhost:8080", email="info@openmined.org", password="changethis"
)

In [ ]:
# third party
from helpers import create_user
from helpers import load_users
from helpers import make_user
from helpers import save_users

In [ ]:
num_users = 2
users = []
email_disable_index = 0
for i in range(num_users):
    user = make_user()
    create_user(high_client, user)
    user.client = high_client
    if email_disable_index == i:
        user.email_disabled = True
    users.append(user)

In [ ]:
save_users(users)

In [ ]:
user = users[0]

In [ ]:
# high_client.api.notifications.deactivate()

In [ ]:
# syft absolute

In [ ]:
# user.client.api.services.user.disable_notifications(

In [ ]:
# user.client.api.notifications.deactivate()

In [ ]:
load_users()